## Lectura del fichero de estado de vacunación publicado por Sanidad en  
* https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/vacunaCovid19.htm


In [1]:
# importamos pandas, numpy y datetime
# previamente ha sido necesatio instalar el motor odfpy con un pip3 install odfpy
import pandas as pd
import numpy as np
from datetime import datetime,timedelta

Version branch cambios 18 enero

preparamos nombre del fichero csv, dentro del directorio csv. Preparamos dos ficheros:
  * `estado_vacunacion_.csv` , última versión del mismo con el acumulado de todos los informes publicados
  * `estado_vacunacion_yearmonthday.csv`, con la versión del día en cuestión, solo con la información de ese día

In [2]:
path_directorio='csv/'
nombre_root_fichero = 'estado_vacunacion_{}'
fecha_informe = datetime.today()
#en el caso de queramos bajar otra fecha diferente a 'hoy'
#fecha_informe = datetime(year=2021,month=1,day=8)
fecha_informe_str=fecha_informe.strftime('%Y%m%d') #fecha formateada para nombre fichero
fecha_informe_pandas = fecha_informe.strftime('%-d/%-m/%y') #idem para el campo pandas
nombre_fichero_con_fecha = path_directorio + nombre_root_fichero.format(fecha_informe_str)
nombre_fichero_acumulado = path_directorio + nombre_root_fichero.format('')

Enlace al fichero en red en formato ods

In [3]:
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_{}.ods'
enlace = enlace_bruto.format(fecha_informe_str)

Leemos el fichero

In [4]:
try:
    datos_row = pd.read_excel(enlace)
    print ('Ok, descargado {}'.format(enlace))
except:
    print ('No esta listo aun')

Ok, descargado https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_20210203.ods


Le echamos un vistazo para asegurar que ha ido todo bien

In [5]:
datos_row

,Unnamed: 0,Dosis entregadas Pfizer (1),Dosis entregadas Moderna (1),Total Dosis entregadas (1),Dosis administradas (2),% sobre entregadas,Nº Personas vacunadas(pauta completada),Fecha de la última vacuna registrada (2)
0,Andalucía,400230,16600,416830,329258,0.789910,104529,2021-02-02
1,Aragón,68175,2900,71075,61036,0.858755,15157,2021-02-02
2,Asturias,69935,2900,72835,61483,0.844141,22227,2021-02-02
3,Baleares,35460,1500,36960,26240,0.709957,5262,2021-02-02
4,Canarias,79530,3200,82730,70794,0.855723,17129,2021-02-02
5,Cantabria,33495,1300,34795,26718,0.767869,5861,2021-02-02
6,Castilla y Leon,149475,6200,155675,132496,0.851106,40565,2021-02-02
7,Castilla La Mancha,101655,4200,105855,92702,0.875745,24525,2021-02-02
8,Cataluña,345770,14300,360070,273724,0.760197,80387,2021-02-02
9,C. Valenciana,175690,7000,182690,162563,0.889830,58452,2021-02-02


..hacemos una copia para hacerle las transformaciones necesarias:

In [6]:
datos_hoy = datos_row.copy()

Primero renombramos columnas, añadimos luego una columna con la fecha del informe `date_pub`, y finalmente añadimos fuente y origen de datos para asegurar trazabilidad de los datos:

In [7]:
datos_hoy.columns=['ccaa','Dosis entregadas Pfizer','Dosis entregadas Moderna','Dosis entregadas','Dosis administradas','% sobre entregadas','Total pauta completada','Fecha de la ultima vacuna registrada']

In [8]:
datos_hoy['date_pub']=fecha_informe_pandas
datos_hoy['source_name']='Ministerio de Sanidad'
datos_hoy['source'] = enlace


vamos a convertir el formato de `Fecha de la ultima vacuna registrada` para que aparezca como día/mes/año sin añadir ceros de pad..de ahi el signo menos en el caso de dia y mes (%-d/%-m). La excepción se añade para la línea de Totales, que no lleva fecha asociada:

In [9]:
def convierte(x):
    campo =  x['Fecha de la ultima vacuna registrada']
    try:
        fecha = campo.strftime('%-d/%-m/%y')
        return fecha
    except:
        return campo

In [10]:
datos_hoy['Fecha de la ultima vacuna registrada'] = datos_hoy.apply(convierte,axis=1)

..y ordenamos..

In [11]:
datos_hoy=datos_hoy[['date_pub','ccaa','Dosis entregadas Pfizer',
                     'Dosis entregadas Moderna','Dosis entregadas',
                     'Dosis administradas','% sobre entregadas',
                     'Total pauta completada',
                     'Fecha de la ultima vacuna registrada',
                     'source_name','source']]

Hemos visto algún informe en el que se añaden espacios al nombre de Comunidad, nos aseguramos aquí de eliminarlos en caso de que vuelvan a aparecer:

In [12]:
datos_hoy['ccaa'] = datos_hoy['ccaa'].apply(lambda x : x.rstrip().lstrip())

In [13]:
datos_hoy['ccaa'].unique()

array(['Andalucía', 'Aragón', 'Asturias', 'Baleares', 'Canarias',
       'Cantabria', 'Castilla y Leon', 'Castilla La Mancha', 'Cataluña',
       'C. Valenciana', 'Extremadura', 'Galicia', 'La Rioja', 'Madrid',
       'Murcia', 'Navarra', 'País Vasco', 'Ceuta', 'Melilla', 'Totales'],
      dtype=object)

In [14]:
datos_hoy

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Fecha de la ultima vacuna registrada,source_name,source
0,3/2/21,Andalucía,400230,16600,416830,329258,0.789910,104529,2/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,3/2/21,Aragón,68175,2900,71075,61036,0.858755,15157,2/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,3/2/21,Asturias,69935,2900,72835,61483,0.844141,22227,2/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,3/2/21,Baleares,35460,1500,36960,26240,0.709957,5262,2/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,3/2/21,Canarias,79530,3200,82730,70794,0.855723,17129,2/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,3/2/21,Cantabria,33495,1300,34795,26718,0.767869,5861,2/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,3/2/21,Castilla y Leon,149475,6200,155675,132496,0.851106,40565,2/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,3/2/21,Castilla La Mancha,101655,4200,105855,92702,0.875745,24525,2/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,3/2/21,Cataluña,345770,14300,360070,273724,0.760197,80387,2/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,3/2/21,C. Valenciana,175690,7000,182690,162563,0.889830,58452,2/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


guardamos el fichero de hoy en el directorio csv

In [15]:
datos_hoy.to_csv(nombre_fichero_con_fecha+'.csv',index = False)

y ahora bajamos el acumulado actual para añadirle los datos de hoy, y volverlo a subir 

In [16]:
datos_acumulados = pd.read_csv(nombre_fichero_acumulado+'.csv')

In [17]:
datos_acumulados

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Última fecha de actualización de datos,Fecha de la ultima vacuna registrada,source_name,source
0,4/1/21,Andalucía,NaN,NaN,140295,25809,0.183962,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 19,https://www.mscbs.gob.es/profesionales/saludPu...
1,4/1/21,Aragón,NaN,NaN,23715,2004,0.084503,NaN,2/1/21,2/1/21,Ministerio de Sanidad - Vacuna Covid 20,https://www.mscbs.gob.es/profesionales/saludPu...
2,4/1/21,Asturias,NaN,NaN,23720,9380,0.395447,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 21,https://www.mscbs.gob.es/profesionales/saludPu...
3,4/1/21,Baleares,NaN,NaN,8940,153,0.017114,NaN,28/12/20,28/12/20,Ministerio de Sanidad - Vacuna Covid 22,https://www.mscbs.gob.es/profesionales/saludPu...
4,4/1/21,Canarias,NaN,NaN,20835,4846,0.232589,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 23,https://www.mscbs.gob.es/profesionales/saludPu...
...,...,...,...,...,...,...,...,...,...,...,...,...
415,2/2/21,Navarra,27645.0,500.0,28145,26779,0.951466,6320.0,NaN,1/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
416,2/2/21,País Vasco,74700.0,1400.0,76100,59582,0.782943,14059.0,NaN,1/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
417,2/2/21,Ceuta,1980.0,200.0,2180,1950,0.894495,748.0,NaN,1/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
418,2/2/21,Melilla,1980.0,200.0,2180,2348,1.077064,1040.0,NaN,1/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


concatenamos...

In [18]:
datos_acumulados=pd.concat([datos_acumulados,datos_hoy],ignore_index=True)

In [19]:
datos_acumulados.columns

Index(['date_pub', 'ccaa', 'Dosis entregadas Pfizer',
       'Dosis entregadas Moderna', 'Dosis entregadas', 'Dosis administradas',
       '% sobre entregadas', 'Total pauta completada',
       'Última fecha de actualización de datos',
       'Fecha de la ultima vacuna registrada', 'source_name', 'source'],
      dtype='object')

In [20]:
datos_acumulados = datos_acumulados[['date_pub', 'ccaa','Dosis entregadas Pfizer','Dosis entregadas Moderna',
                                     'Dosis entregadas', 'Dosis administradas','% sobre entregadas',
                                     'Total pauta completada','Última fecha de actualización de datos',
                                     'Fecha de la ultima vacuna registrada','source_name', 'source']]

In [21]:
datos_acumulados

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Última fecha de actualización de datos,Fecha de la ultima vacuna registrada,source_name,source
0,4/1/21,Andalucía,NaN,NaN,140295,25809,0.183962,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 19,https://www.mscbs.gob.es/profesionales/saludPu...
1,4/1/21,Aragón,NaN,NaN,23715,2004,0.084503,NaN,2/1/21,2/1/21,Ministerio de Sanidad - Vacuna Covid 20,https://www.mscbs.gob.es/profesionales/saludPu...
2,4/1/21,Asturias,NaN,NaN,23720,9380,0.395447,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 21,https://www.mscbs.gob.es/profesionales/saludPu...
3,4/1/21,Baleares,NaN,NaN,8940,153,0.017114,NaN,28/12/20,28/12/20,Ministerio de Sanidad - Vacuna Covid 22,https://www.mscbs.gob.es/profesionales/saludPu...
4,4/1/21,Canarias,NaN,NaN,20835,4846,0.232589,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 23,https://www.mscbs.gob.es/profesionales/saludPu...
...,...,...,...,...,...,...,...,...,...,...,...,...
435,3/2/21,Navarra,33495.0,1300.0,34795,28199,0.810433,7148.0,NaN,2/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
436,3/2/21,País Vasco,91080.0,3600.0,94680,62679,0.662009,16621.0,NaN,2/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
437,3/2/21,Ceuta,1980.0,200.0,2180,2008,0.921101,806.0,NaN,2/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
438,3/2/21,Melilla,1980.0,200.0,2180,2372,1.088073,1062.0,NaN,2/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [22]:
datos_acumulados['ccaa'].unique()

array(['Andalucía', 'Aragón', 'Asturias', 'Baleares', 'Canarias',
       'Cantabria', 'Castilla y Leon', 'Castilla La Mancha', 'Cataluña',
       'C. Valenciana', 'Extremadura', 'Galicia', 'La Rioja', 'Madrid',
       'Murcia', 'Navarra', 'País Vasco', 'Ceuta', 'Melilla', 'Totales'],
      dtype=object)

y volvemos a subirlo..

In [23]:
datos_acumulados.to_csv(nombre_fichero_acumulado+'.csv',index=False)